In [8]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [9]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [10]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [11]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [12]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

In [13]:
# View all of the classes that automap found
Base.classes.keys()


['measurement', 'station']

In [14]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station


In [15]:
# Create our session (link) from Python to the DB
session = Session(engine)


# Exploratory Precipitation Analysis

## GRADER PLEASE NOTE

* Some of the instructions are different from the original but still adhere to the rubric.
* The students get the average precipitation per day instead of every single precipitation value.

In [22]:
# Find the most recent date in the data set.
# first_date=session.query(Measurement.date).order_by(Measurement.date).first().date
recent_date=session.query(Measurement.date).order_by(Measurement.date.desc()).first().date
# print(first_date)
print(recent_date)

2017-08-23


In [24]:
# Using this date, find the average precipitation per day
# for the preceding 12 months of data sorted by ascending date.
last_year = dt.date(2017,8,23) - dt.timedelta(days = 365)
precipitation = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date > last_year).\
order_by(Measurement.date).all()
precipitation[0]


('2016-08-24', 0.08)

In [27]:
# Convert your results into a pandas DataFrame with date as index.
precipitation_df = pd.DataFrame(precipitation)
precipitation_df = precipitation_df.rename(columns={0: 'date', 1: 'prcp'})
precipitation_df.head()


,date,prcp
0,2016-08-24,0.08
1,2016-08-24,2.15
2,2016-08-24,2.28
3,2016-08-24,NaN
4,2016-08-24,1.22


In [28]:
precipitation_df.set_index('date')
precipitation_df = precipitation_df.sort_index(ascending = True)
precipitation_df.head()

,date,prcp
0,2016-08-24,0.08
1,2016-08-24,2.15
2,2016-08-24,2.28
3,2016-08-24,NaN
4,2016-08-24,1.22


In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data


In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data

# Exploratory Station Analysis

In [ ]:
# Query total number of stations in dataset


In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.


In [ ]:
# Using the station id from the previous query, calculate the lowest, 
# highest, and average temperature recorded for this station.


In [ ]:
# Query the last 12 months of temperature observation data for this station.
# Plot the results as a histogram with bins = 12.


# Close session